# Μηχανή Αναζήτησης με βάση την Wikepedia

Αυτό το Notebook περιγράφει τη διαδικασία σχεδιασμού, υλοποίησης και αξιολόγησης μιας απλής μηχανής αναζήτησης με βάση την Ανάκτηση Πληροφορίας. Θα δούμε βήμα-βήμα πώς επεξεργαστήκαμε τα δεδομένα, υλοποιήσαμε αλγόριθμους αναζήτησης και αξιολογήσαμε την απόδοσή τους.

## Προετοιμασία

Πριν ξεκινήσουμε να εκτελούμε τα ζητούμενα της άσησης, πρέπει να κάνουμε τα κατάληλλα installs, οπως είναι το nltk και το BeautifulSoup, το οποιο θα χρησημοποιηθεί στο πρώτο βήμα.

In [ ]:
pip install nltk

In [ ]:
pip install beautifulsoup4 requests

## Βημα 1

Στην συνέχεια πραγματαποιούμαι εναν web-crawler με την χρήση της βιβλιοθήκης Beautiful Soup. Ετσι βρίσκουμε, με την βοήθεια keywords, και αποθηκεύουμε 20 σχετικά άρθρα για περεταίρω ανάλυση.
Παρακάτω φαίνεται η εκτέλεση αυτών:

In [3]:
import requests
from bs4 import BeautifulSoup
import json

def get_wikipedia_articles(base_url, keywords, total_articles=20):
    articles = []
    visited_urls = set()  # Χρησιμοποιείται για την αποφυγή διπλότυπων
    article_count = 0

    for keyword in keywords:
        #if article_count >= total_articles:
          #  break

        search_url = f"{base_url}/w/index.php?search={keyword}"
        print(f"Searching articles for keyword: {keyword}")
        response = requests.get(search_url)
        soup = BeautifulSoup(response.content, "html.parser")

        # Εύρεση συνδέσμων άρθρων
        links = soup.select("a[href^='/wiki/']")
        for link in links:
            if article_count >= total_articles:
                break

            href = link.get('href')
            if not href.startswith('/wiki/') or ':' in href:  # Αγνοήστε ειδικές σελίδες
                continue

            article_url = base_url + href
            if article_url in visited_urls:  # Αν έχει ήδη επισκεφθεί
                continue

            visited_urls.add(article_url)  # Προσθέστε στο σύνολο
            article_response = requests.get(article_url)
            article_soup = BeautifulSoup(article_response.content, "html.parser")

            # Εξαγωγή τίτλου και περιεχομένου
            title = article_soup.find("h1").text
            paragraphs = article_soup.find_all("p")
            content = " ".join([p.text for p in paragraphs])

            print(f"Found article #{article_count + 1}: {title}")
            articles.append({
                "id": article_count + 1,  # Αριθμός άρθρου
                "title": title,
                "content": content,
                "url": article_url
            })
            article_count += 1

    return articles

# Παράδειγμα χρήσης
if __name__ == "__main__":
    base_url = "https://en.wikipedia.org"
    keywords = ["machine learning", "data science", "artificial intelligence", "neural networks", "deep learning"]
    data = get_wikipedia_articles(base_url, keywords, total_articles=20)

    # Αποθήκευση σε JSON
    with open("wikipedia_articles.json", "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

    print("Συλλογή ολοκληρώθηκε και αποθηκεύτηκε σε αρχείο JSON.")


Searching articles for keyword: machine learning
Found article #1: Main Page
Found article #2: Machine learning
Found article #3: Machine Learning (journal)
Found article #4: Statistical learning in language acquisition
Found article #5: Data mining
Found article #6: Supervised learning
Found article #7: Unsupervised learning
Found article #8: Weak supervision
Found article #9: Self-supervised learning
Found article #10: Reinforcement learning
Found article #11: Meta-learning (computer science)
Found article #12: Online machine learning
Found article #13: Online machine learning
Found article #14: Curriculum learning
Found article #15: Rule-based machine learning
Found article #16: Neuro-symbolic AI
Found article #17: Neuromorphic computing
Found article #18: Quantum machine learning
Found article #19: Statistical classification
Found article #20: Generative model
Searching articles for keyword: data science
Searching articles for keyword: artificial intelligence
Searching articles for

## Βήμα 2

Για την προεπεξεργασία εφαρμόζουμε: tokenization, αφαίρεση stop-words, και stemming/lemmatization. Αυτά τα βήματα βοηθούν στη μείωση του μεγέθους του λεξιλογίου και στη βελτίωση της απόδοσης.

In [5]:
import json
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Κατεβάστε δεδομένα για NLTK (αν χρειάζεται)
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

def preprocess_text(text):
    # 1. Αφαίρεση ειδικών χαρακτήρων
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # 2. Μετατροπή σε πεζά
    text = text.lower()

    # 3. Tokenization
    tokens = word_tokenize(text)

    # 4. Αφαίρεση stop-words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # 5. Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return tokens

def preprocess_dataset(input_file, output_file):
    # Φόρτωση δεδομένων από το JSON
    with open(input_file, "r", encoding="utf-8") as file:
        data = json.load(file)

    # Καθαρισμός περιεχομένου
    for article in data:
        article['processed_content'] = preprocess_text(article['content'])

    # Αποθήκευση καθαρισμένων δεδομένων σε νέο αρχείο
    with open(output_file, "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

    print(f"Τα δεδομένα προεπεξεργάστηκαν και αποθηκεύτηκαν στο {output_file}")

# Παράδειγμα χρήσης
if __name__ == "__main__":
    preprocess_dataset("wikipedia_articles.json", "processed_wikipedia_articles.json")


Τα δεδομένα προεπεξεργάστηκαν και αποθηκεύτηκαν στο processed_wikipedia_articles.json


Ακολουθούν οι πρώτες 20 γραμμες του .json αρχείου για να δειχθεί η μορφή και να παραμείνει ευανάγνωστο το αρχείο. 

In [ ]:
[
    {
        "id": 1,
        "title": "Main Page",
        "content": "January 19\n The roadside hawk (Rupornis magnirostris) is a relatively small bird of prey in the family Accipitridae. It is found from Mexico through Central America and in most of South America east of the Andes. With the possible exception of dense rainforests, the roadside hawk is well adapted to most ecosystems in its range. It is also an urban bird, and is possibly the most common species of hawk seen in various cities throughout its range. This roadside hawk of the subspecies R. m. griseocauda was photographed feeding on a speckled racer in Orange Walk District, Belize.\n Photograph credit: Charles J. Sharp Wikipedia is written by volunteer editors and hosted by the Wikimedia Foundation, a non-profit organization that also hosts a range of other volunteer projects:\n This Wikipedia is written in English. Many other Wikipedias are available; some of the largest are listed below.\n",
        "url": "https://en.wikipedia.org/wiki/Main_Page",
        "processed_content": [
            "january",
            "roadside",
            "hawk",
            "rupornis",
            "magnirostris",
            "relatively",
            "small",
            "bird",
            "prey",
            "family",
            "accipitridae",
            "found",
            "mexico",

## Βήμα 3

Δημιουργούμε ένα ανεστραμμένο ευρετήριο που αποθηκεύει τους όρους και τα εγγραφα στα οποία εμφανίζεται ο κάθε όρος.

In [6]:
import json
from collections import defaultdict

def create_inverted_index(input_file, output_file):
    # Φόρτωση επεξεργασμένων δεδομένων
    with open(input_file, "r", encoding="utf-8") as file:
        data = json.load(file)

    # Ανεστραμμένο ευρετήριο
    inverted_index = defaultdict(list)

    for article in data:
        doc_id = article["id"]
        processed_content = article["processed_content"]

        for word in processed_content:
            if doc_id not in inverted_index[word]:
                inverted_index[word].append(doc_id)

    # Αποθήκευση του ανεστραμμένου ευρετηρίου
    with open(output_file, "w", encoding="utf-8") as file:
        json.dump(inverted_index, file, ensure_ascii=False, indent=4)

    print(f"Το ανεστραμμένο ευρετήριο αποθηκεύτηκε στο {output_file}")

# Παράδειγμα χρήσης
if __name__ == "__main__":
    create_inverted_index("processed_wikipedia_articles.json", "inverted_index.json")



Το ανεστραμμένο ευρετήριο αποθηκεύτηκε στο inverted_index.json


Παραθέτονται παραδείγματα απο το inverted_index.json

In [ ]:
"listed": [
        1
    ],
    "machine": [
        2,
        3,
        5,
        6,
        7,
        8,
        9,
        10,
        11,
        12,
        13,
        14,
        15,
        16,
        17,
        18,
        19
    ],
    "learning": [
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        11,
        12,
        13,
        14,
        15,
        16,
        17,
        18,
        19,
        20
    ],

## Βήμα 4.α

Υλοποιούμε μια μηχανή αναζήτησης για απλά ερωτήματα και boolean εντολές.

In [ ]:
import json
from functools import reduce

def load_data(index_file, articles_file):
    with open(index_file, "r", encoding="utf-8") as file:
        inverted_index = json.load(file)
    with open(articles_file, "r", encoding="utf-8") as file:
        articles = {article["id"]: article for article in json.load(file)}
    return inverted_index, articles

def process_query(query):
    tokens = query.lower().split()
    processed_query = []
    operators = {"and", "or", "not"}
    for token in tokens:
        if token in operators:
            processed_query.append(token.upper())
        else:
            processed_query.append(token)
    return processed_query

def search(articles,inverted_index, query_tokens):
    result_stack = []
    all_doc_ids = set(articles.keys())  # Όλα τα έγγραφα στο ευρετήριο
   
    operators = []
    operands = []

    print(f"Αναζήτηση για το ερώτημα: {' '.join(query_tokens)}")

    for token in query_tokens:
   

        if token == "AND":
            operators.append(token)  
        elif token == "OR":
            operators.append(token)  
        elif token == "NOT":
            operators.append(token)  
        else:
            # Προσθέτουμε τον όρο στη στοίβα αν υπάρχει στο ευρετήριο
            if token in inverted_index:
                operands.append(set(inverted_index[token]))  
            else:
                operands.append(set())  
                print(f"Ο όρος '{token}' δεν βρέθηκε στο ευρετήριο, προστέθηκε κενό σύνολο.")

       
    # Επεξεργασία των τελεστών μετά την προσθήκη όλων των όρων
    while operators:
        operator = operators.pop(0)  
        if operator == "AND":
            operand2 = operands.pop(0)  
            operand1 = operands.pop(0)
            operands.insert(0, operand1 & operand2)  # Τομή
        elif operator == "OR":
            operand2 = operands.pop(0)
            operand1 = operands.pop(0)
            operands.insert(0, operand1 | operand2)  # Ένωση
        elif operator == "NOT":
            operand1 = operands.pop(0)
            result = all_doc_ids - operand1
            print(f"Result after NOT: {result}")
            operands.insert(0, result) 
            print(f"Εφαρμόστηκε NOT, το αποτέλεσμα είναι: {result}")
    
    if len(operands) != 1:
        print("Σφάλμα: Το ερώτημα δεν επεξεργάστηκε σωστά.")
        return []

 
    return operands.pop()

def display_results(results, articles):
    if not results:
        print("Δεν βρέθηκαν σχετικά έγγραφα.")
    else:
        print(f"Βρέθηκαν {len(results)} έγγραφα:")
        for doc_id in results:
            print(f"- ID: {doc_id}, Title: {articles[doc_id]['title']}")

def main():
    index_file = "inverted_index.json"
    articles_file = "processed_wikipedia_articles.json"

    inverted_index, articles = load_data(index_file, articles_file)

    print("Καλώς ήρθατε στη Μηχανή Αναζήτησης!")
    print("Χρησιμοποιήστε λογικούς τελεστές (AND, OR, NOT) για πιο σύνθετα ερωτήματα.")
    
    while True:
        query = input("Εισάγετε το ερώτημά σας (ή 'exit' για έξοδο): ").strip()
        if query.lower() == "exit":
            print("Ευχαριστούμε που χρησιμοποιήσατε τη μηχανή αναζήτησης!")
            break

        query_tokens = process_query(query)

        results = search(articles,inverted_index, query_tokens)
        display_results(results, articles)

if __name__ == "__main__":
    main()


Καλώς ήρθατε στη Μηχανή Αναζήτησης!
Χρησιμοποιήστε λογικούς τελεστές (AND, OR, NOT) για πιο σύνθετα ερωτήματα.


Θα υπάρχει screenshot της ολοκληρωμένης εκτέλεσης στο έγγραφο τεκμηρίωσης, καθώς χρειάζεται user input. Παρακάτων φαίνεται αντιγραμμένη η έξοδος.

Καλώς ήρθατε στη Μηχανή Αναζήτησης!
Χρησιμοποιήστε λογικούς τελεστές (AND, OR, NOT) για πιο σύνθετα ερωτήματα.
Εισάγετε το ερώτημά σας (ή 'exit' για έξοδο): machine
Αναζήτηση για το ερώτημα: machine
Βρέθηκαν 17 έγγραφα:
- ID: 2, Title: Machine learning
- ID: 3, Title: Machine Learning (journal)
- ID: 5, Title: Data mining
- ID: 6, Title: Supervised learning
- ID: 7, Title: Unsupervised learning
- ID: 8, Title: Weak supervision
- ID: 9, Title: Self-supervised learning
- ID: 10, Title: Reinforcement learning
- ID: 11, Title: Meta-learning (computer science)
- ID: 12, Title: Online machine learning
- ID: 13, Title: Online machine learning
- ID: 14, Title: Curriculum learning
- ID: 15, Title: Rule-based machine learning
- ID: 16, Title: Neuro-symbolic AI
- ID: 17, Title: Neuromorphic computing
- ID: 18, Title: Quantum machine learning
- ID: 19, Title: Statistical classification
Εισάγετε το ερώτημά σας (ή 'exit' για έξοδο): exit
Ευχαριστούμε που χρησιμοποιήσατε τη μηχανή αναζήτησης!

## Βήμα 4.β

Ταξινομούμε τις εμφανίσεις της δοθείσας λέξης με βάση τους αντίστοιχους αλγορίθμους.

In [ ]:
import json
import numpy as np  # type: ignore
from sklearn.feature_extraction.text import TfidfVectorizer  # type: ignore
from rank_bm25 import BM25Okapi  # type: ignore

# Load the data (articles)
def load_data(articles_file):
    with open(articles_file, "r", encoding="utf-8") as file:
        articles = {article["id"]: article for article in json.load(file)}
    return articles

# Boolean Retrieval
def search_boolean(query, documents):
    query_terms = set(query.lower().split())
    doc_scores = {}

    for doc_idx, content in enumerate(documents):
        terms_in_doc = set(content.lower().split())
        if query_terms.issubset(terms_in_doc):  # All query terms must be in the document
            doc_scores[doc_idx] = sum(content.lower().split().count(term) for term in query_terms)

    ranked_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
    return [doc[0] for doc in ranked_docs], [doc[1] for doc in ranked_docs]

# TF-IDF Ranking


def rank_with_tfidf(query, documents):
    """
    Rank documents using TF-IDF and cosine similarity to the query.

    Args:
        query (str): The user's search query.
        documents (list of str): The list of document contents.

    Returns:
        ranked_docs (list of int): Indices of documents sorted by relevance.
        scores (list of float): Corresponding cosine similarity scores.
    """
    # Create a TF-IDF vectorizer and fit on the documents
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(documents)

    # Transform the query into the same TF-IDF space
    query_vector = vectorizer.transform([query])

    # Compute cosine similarity between the query vector and all document vectors
    cosine_similarities = (tfidf_matrix @ query_vector.T).toarray().flatten()

    # Rank documents by descending similarity
    ranked_docs = np.argsort(cosine_similarities)[::-1]
    scores = cosine_similarities[ranked_docs]

    return ranked_docs, scores



# BM25 Ranking
def rank_with_bm25(query, documents):
    tokenized_docs = [doc.lower().split() for doc in documents]
    tokenized_query = query.lower().split()

    bm25 = BM25Okapi(tokenized_docs)
    doc_scores = bm25.get_scores(tokenized_query)
    ranked_docs = np.argsort(doc_scores)[::-1]
    scores = [doc_scores[idx] for idx in ranked_docs]
    return ranked_docs, scores

def display_ranked_results(ranked_docs, scores, articles):
    # Ensure results with non-zero scores are displayed
    has_results = False
    valid_results = [(doc_idx, score) for doc_idx, score in zip(ranked_docs, scores) if score != 0]


    print(f"Βρέθηκαν {len(valid_results)} σχετικά έγγραφα:")
    for i, (doc_idx, score) in enumerate(zip(ranked_docs, scores)):
        if score != 0:  # Only display documents with a non-zero score
            has_results = True
            doc_id = list(articles.keys())[doc_idx]  # Map index to document ID
            print(f"- ID: {doc_id}, Score: {score:.4f}, Title: {articles[doc_id]['title']}")

    if not has_results:
        print("Δεν βρέθηκαν σχετικά έγγραφα.")


def display_boolean_results(ranked_docs, scores, articles):
    if not ranked_docs:  # Check if the list is empty
        print("Δεν βρέθηκαν σχετικά έγγραφα.")
        return

    print(f"Βρέθηκαν {len(ranked_docs)} έγγραφα:")
    for i, doc_idx in enumerate(ranked_docs):
        doc_id = list(articles.keys())[doc_idx]
        print(f"- ID: {doc_id}, Occurrences: {scores[i]}, Title: {articles[doc_id]['title']}")


# Main function
def main():
    articles_file = "processed_wikipedia_articles.json"
    articles = load_data(articles_file)
    documents = [article["content"] for article in articles.values()]

    print("Καλώς ήρθατε στη Μηχανή Αναζήτησης!")
    print("Επιλέξτε τον αλγόριθμο ανάκτησης:")
    print("1. Boolean Retrieval")
    print("2. TF-IDF")
    print("3. BM25")

    while True:
        query = input("Εισάγετε το ερώτημά σας (ή 'exit' για έξοδο): ").strip()
        if query.lower() == "exit":
            print("Ευχαριστούμε που χρησιμοποιήσατε τη μηχανή αναζήτησης!")
            break

        algo_choice = input("Επιλέξτε αλγόριθμο (1/2/3): ").strip()
        if algo_choice == '1':
            print("\nΧρησιμοποιώντας Boolean Retrieval για την κατάταξη...\n")
            ranked_docs, scores = search_boolean(query, documents)
            display_boolean_results(ranked_docs, scores, articles)
        elif algo_choice == '2':
            print("\nΧρησιμοποιώντας TF-IDF για την κατάταξη...\n")
            ranked_docs, scores = rank_with_tfidf(query, documents)
            display_ranked_results(ranked_docs, scores, articles)
        elif algo_choice == '3':
            print("\nΧρησιμοποιώντας BM25 για την κατάταξη...\n")
            ranked_docs, scores = rank_with_bm25(query, documents)
            display_ranked_results(ranked_docs, scores, articles)
        else:
            print("Λανθασμένη επιλογή. Παρακαλώ δοκιμάστε ξανά.")

if __name__ == "__main__":
    main()


Καλώς ήρθατε στη Μηχανή Αναζήτησης!
Επιλέξτε τον αλγόριθμο ανάκτησης:
1. Boolean Retrieval
2. TF-IDF
3. BM25


Όπως και στο προηγούμενο υποερώτημα, θα υπάρχουν screenshot στο έγγραφο τεκμηρίωσης και εδώ φαίνεται αντιγραμμένη μία ενδεικτική είσοδος

Εισάγετε το ερώτημά σας (ή 'exit' για έξοδο): machine
Επιλέξτε αλγόριθμο (1/2/3): 1

Χρησιμοποιώντας Boolean Retrieval για την κατάταξη...

Βρέθηκαν 17 έγγραφα:
- ID: 2, Occurrences: 110, Title: Machine learning
- ID: 18, Occurrences: 47, Title: Quantum machine learning
- ID: 5, Occurrences: 8, Title: Data mining
- ID: 15, Occurrences: 7, Title: Rule-based machine learning
- ID: 12, Occurrences: 5, Title: Online machine learning
- ID: 13, Occurrences: 5, Title: Online machine learning
- ID: 7, Occurrences: 4, Title: Unsupervised learning
- ID: 10, Occurrences: 4, Title: Reinforcement learning
- ID: 3, Occurrences: 3, Title: Machine Learning (journal)
- ID: 11, Occurrences: 3, Title: Meta-learning (computer science)
- ID: 14, Occurrences: 3, Title: Curriculum learning
- ID: 8, Occurrences: 2, Title: Weak supervision
- ID: 9, Occurrences: 2, Title: Self-supervised learning
- ID: 17, Occurrences: 2, Title: Neuromorphic computing
- ID: 6, Occurrences: 1, Title: Supervised learning
- ID: 16, Occurrences: 1, Title: Neuro-symbolic AI
- ID: 19, Occurrences: 1, Title: Statistical classification